# Funções para recomendações

In [2]:
from math import sqrt
from recomendacoes import *

# Carregando o dataframe
def df_movie_lens(path='ml-100k'):
    # Inicializa dicionário de filmes
    filmes = {}
    for linha in open(path + '/u.item', encoding='latin-1'):
        (id, titulo) = linha.split('|')[0:2]
        filmes[id] = titulo

    base = {}
    # Percorre os usuários e armazena no dicionário
    for linha in open(path + '/u.data', encoding='latin-1'):
        (usuario, idfilme, nota, tempo) = linha.split('\t')
        base.setdefault(usuario, {})
        base[usuario][filmes[idfilme]] = float(nota)

    return base

def euclidiana(base, usuario1, usuario2):
    si = {item: 1 for item in base[usuario1] if item in base[usuario2]}

    if len(si) == 0:
        return 0  # Nenhuma similaridade se não houver itens em comum

    soma = sum([pow(base[usuario1][item] - base[usuario2][item], 2) for item in si])
    return 1 / (1 + sqrt(soma))

def users_similares(base, usuario):
    similaridade = [(euclidiana(base, usuario, outro), outro) for outro in base if outro != usuario]
    similaridade.sort()
    similaridade.reverse()
    return similaridade[0:10]

def recomenda_users(base, usuario):
    totais = {}
    somaSimilaridade = {}

    for outro in base:
        if outro == usuario:
            continue
        similaridade = euclidiana(base, usuario, outro)

        if similaridade <= 0:
            continue

        for item in base[outro]:
            if item not in base[usuario]:
                totais.setdefault(item, 0)
                totais[item] += base[outro][item] * similaridade
                somaSimilaridade.setdefault(item, 0)
                somaSimilaridade[item] += similaridade

    rankings = [(total / somaSimilaridade[item], item) for item, total in totais.items()]
    rankings.sort()
    rankings.reverse()
    return rankings[0:30]

def calcula_itens_similares(base):
    result = {}
    for item in base:
        notas = users_similares(base, item)
        result[item] = notas
    return result

itens_similares = calcula_itens_similares(avaliacoes_filmes)

def recomenda_itens(base_usuario, similaridade_itens, usuario):
    notasUsuario = base_usuario[usuario]
    notas = {}
    totalSimilaridade = {}

    for (item, nota) in notasUsuario.items():
        for (similaridade, item2) in similaridade_itens[item]:
            if item2 in notasUsuario:
                continue
            notas.setdefault(item2, 0)
            notas[item2] += similaridade * nota
            totalSimilaridade.setdefault(item2, 0)
            totalSimilaridade[item2] += similaridade

    rankings = [(score / totalSimilaridade[item], item) for item, score in notas.items()]
    rankings.sort()
    rankings.reverse()
    return rankings